In [1]:
# Importação das Bobliotecas Necessárias

import requests
# Importação das Boblioteca de Dados
import pandas as pd
import numpy as np
# Importação das Boblioteca de Plotagem
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Importação das Boblioteca de Modelos Estatisticos
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
# Ignorando Avisos
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#Buscando dados das API's

api_despesas = 'http://www.al.sp.gov.br/repositorioDados/deputados/despesas_gabinetes.xml'
api_cadastro = 'http://www.al.sp.gov.br/repositorioDados/deputados/deputados.xml'

#fazendo a requisição
conexao_despesas = requests.get(api_despesas)
conexao_cadastro = requests.get(api_cadastro)

#testando conexão com as APIs

print(conexao_cadastro, conexao_despesas)

<Response [200]> <Response [200]>


In [9]:
#Verificando dados da primeira API Despesas

base_despesas = pd.read_xml( conexao_despesas.content )

#Verificando o Tamanho do Dataset
base_despesas.shape



(528843, 8)

In [7]:
#Verificando a Estrutura e Dtype da API
base_despesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528843 entries, 0 to 528842
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Ano         528843 non-null  int64  
 1   Matricula   528843 non-null  int64  
 2   Mes         528843 non-null  int64  
 3   Valor       528843 non-null  float64
 4   CNPJ        518061 non-null  object 
 5   Deputado    528843 non-null  object 
 6   Tipo        528843 non-null  object 
 7   Fornecedor  528843 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 32.3+ MB


In [8]:
#Analisando as 5 primeiras linhas do Dataset
base_despesas.head()

,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA


In [10]:
#Verificando dados da segunda API cadastro

base_cadastro = pd.read_xml( conexao_cadastro.content )

#Verificando o Tamanho do Dataset
base_cadastro.shape


(93, 16)

In [12]:
#Verificando a Estrutura e Dtype da API
base_cadastro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   IdDeputado       93 non-null     int64 
 1   IdSPL            93 non-null     int64 
 2   IdUA             93 non-null     int64 
 3   Situacao         93 non-null     object
 4   Andar            92 non-null     object
 5   Aniversario      93 non-null     object
 6   Biografia        93 non-null     object
 7   Email            87 non-null     object
 8   Matricula        93 non-null     int64 
 9   NomeParlamentar  93 non-null     object
 10  PathFoto         93 non-null     object
 11  Sala             93 non-null     object
 12  Partido          93 non-null     object
 13  Telefone         93 non-null     object
 14  Fax              13 non-null     object
 15  PlacaVeiculo     1 non-null      object
dtypes: int64(4), object(12)
memory usage: 11.8+ KB


In [13]:
#Analisando as 5 primeiras linhas do Dataset
base_cadastro.head()

,IdDeputado,IdSPL,IdUA,Situacao,Andar,Aniversario,Biografia,Email,Matricula,NomeParlamentar,PathFoto,Sala,Partido,Telefone,Fax,PlacaVeiculo
0,549,1000000330,20377,EXE,3º andar,01/11,"<p style=""margin: 0cm 0cm 0.0001pt; background...",adalbertofreitas@al.sp.gov.br,300605,Adalberto Freitas,http://www3.al.sp.gov.br/repositorio/deputadoP...,305,PSDB,(11) 3886-6557 / 6559,None,None
1,548,1000000332,20370,EXE,Térreo,09/07,"<p class=""MsoNormal"">Com uma hist&oacute;ria d...",adrianaborgo@al.sp.gov.br,300606,Adriana Borgo,http://www3.al.sp.gov.br/repositorio/deputadoP...,T. 45,AGIR,(11) 3886-6567/8770,None,None
2,547,1000000335,20455,EXE,2º andar,26/08,<p>Danilo Mascarenhas de Balas nasceu na cidad...,apfdanilobalas@al.sp.gov.br,300607,Agente Federal Danilo Balas,http://www3.al.sp.gov.br/repositorio/deputadoP...,255,PL,(11)3886-6052/6054,None,None
3,128,39,20369,EXE,3º,04/01,"<p style=""outline: none; color: #121212; margi...",ademarchi@al.sp.gov.br,300288,Aldo Demarchi,http://www3.al.sp.gov.br/repositorio/deputadoP...,356,UNIÃO,3886-6453/6454,None,None
4,546,1000000342,20399,EXE,1º,17/09,"<p>Alexander Muniz de Oliveira, nascido na cid...",alexdemadureira@al.sp.gov.br,300608,Alex de Madureira,http://www3.al.sp.gov.br/repositorio/deputadoP...,173,PL,(11)3886-6676 / 6677,None,None


In [15]:
#

#Filtrar por Deputados Ativos
filtro = base_despesas.loc[(base_despesas['Ano']>= 2019)]


,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
293,2019,300257,1,8000.00,03136828000160,ABELARDO CAMARINHA,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",MARCO ANTONIO D AVILA
294,2019,300257,1,5600.00,06222505000187,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,WAGNER HUBERT EPP
295,2019,300257,1,5500.00,17035632000193,ABELARDO CAMARINHA,F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTOR...,RED VISION PRODUTORA LTDA-ME
296,2019,300257,1,8000.00,11827486000152,ABELARDO CAMARINHA,J - SERVIÇOS DE COMUNICAÇÃO,IZABEL DE FATIMA RAMOS PEREIRA
297,2019,300257,1,1995.00,06222505000187,ABELARDO CAMARINHA,K - LOCAÇÃO DE BENS MÓVEIS,WAGNER HUBERT EPP
...,...,...,...,...,...,...,...,...
526095,2019,300523,3,60.71,04353528815,WELSON GASPARINI,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",NEIDE GONDIM BORGES LUIZ
526096,2019,300523,3,114.34,09621029000165,WELSON GASPARINI,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",MW5 INFORMATICA E SERVIÇOS LTDA. ME
526097,2019,300523,3,163.33,31112521000164,WELSON GASPARINI,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",QUATRO TECH SERVIÇOS E TECNOLOGIA
526098,2019,300523,3,72.83,61216917000112,WELSON GASPARINI,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",CENTRO ELETRONICO E COMERCIAL SETRON LTDA. - EPP


In [18]:
#Agrupar Por ano e por Deputado

analise_ano_deputado = filtro.groupby(by=['Ano', 'Matricula']).sum()['Valor'].reset_index()

#Criando Tabela com anos por coluna e deletando os valores vazios

analise_ano_deputado = analise_ano_deputado.pivot(index='Matricula', columns ='Ano',values = 'Valor').dropna().reset_index()

#Analisando Dados 
analise_ano_deputado

Ano,Matricula,2019,2020,2021,2022
0,300098,253434.48,291386.79,322863.63,166128.97
1,300188,237587.17,247384.14,300364.14,146092.43
2,300205,102936.63,126638.83,84064.04,45873.55
3,300217,302073.11,303710.00,370183.68,177819.15
4,300260,139872.47,125206.46,144298.34,63517.17
...,...,...,...,...,...
83,300646,154261.19,236119.84,333694.06,197470.91
84,300647,173487.95,167221.22,210208.62,106826.84
85,300648,220277.05,234087.89,269953.47,158858.29
86,300649,103766.46,210226.82,287744.27,125486.42


In [24]:
#Cruzando dados dos dataset analise_ano_deputado com o Cadastro

tabela_completa = pd.merge( base_cadastro[['Matricula','NomeParlamentar','Partido','Email','Telefone']],analise_ano_deputado, on='Matricula', how = 'inner')
tabela_completa

,Matricula,NomeParlamentar,Partido,Email,Telefone,2019,2020,2021,2022
0,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,221749.58,254126.85,344173.27,201652.59
1,300606,Adriana Borgo,AGIR,adrianaborgo@al.sp.gov.br,(11) 3886-6567/8770,215277.49,208642.91,179485.53,82590.58
2,300607,Agente Federal Danilo Balas,PL,apfdanilobalas@al.sp.gov.br,(11)3886-6052/6054,180111.95,264914.23,429704.43,162263.06
3,300608,Alex de Madureira,PL,alexdemadureira@al.sp.gov.br,(11)3886-6676 / 6677,252941.28,178229.80,250499.10,116055.05
4,300553,Alexandre Pereira,SD,alexandrepereira@al.sp.gov.br,(11)3886-6426 / 6428,51944.77,79835.09,151268.55,96413.15
...,...,...,...,...,...,...,...,...,...
82,300584,Teonilio Barba,PT,teoniliobarba@al.sp.gov.br,(011)3886-6493 / 6495,397865.74,291635.94,361699.85,200442.14
83,300648,Thiago Auricchio,PL,ThiagoAuricchio@al.sp.gov.br,(011)3886-6214 / 6216,220277.05,234087.89,269953.47,158858.29
84,300649,Valeria Bolsonaro,PL,ValeriaBolsonaro@al.sp.gov.br,(011)3886-6222 / 6223,103766.46,210226.82,287744.27,125486.42
85,300430,Vinícius Camarinha,PSDB,depviniciuscamarinha@al.sp.gov.br,3886-6366/6365/6494,316510.44,302959.16,436087.57,190297.15


In [50]:
analise_gasto_partido_19 = tabela_completa.groupby(by=['Partido']).sum()[2019].reset_index()
analise_gasto_partido_20 = tabela_completa.groupby(by=['Partido']).sum()[2020].reset_index()
analise_gasto_partido_21 = tabela_completa.groupby(by=['Partido']).sum()[2021].reset_index()
analise_gasto_partido_22 = tabela_completa.groupby(by=['Partido']).sum()[2022].reset_index()
#analise_gasto_partido_22.sort_values([2022])

In [58]:
#Analisando os gastos por pardidos nos anos de 2019 até 2022 com a biblioteca plotly
fig = go.Figure()
fig.add_trace(go.Bar(
    x=analise_gasto_partido_19["Partido"],
    y=analise_gasto_partido_19[2019],
    name='Gastos 2019',
    marker_color='Orange'
))
fig.add_trace(go.Bar(
    x=analise_gasto_partido_20["Partido"],
    y=analise_gasto_partido_20[2020],
    name='Gastos 2020',
    marker_color='Red'
))

fig.add_trace(go.Bar(
    x=analise_gasto_partido_21["Partido"],
    y=analise_gasto_partido_21[2021],
    name='Gastos 2021',
    marker_color='Green'
))

fig.add_trace(go.Bar(
    x=analise_gasto_partido_22["Partido"],
    y=analise_gasto_partido_22[2022],
    name='Gastos 2022',
    marker_color='Blue'
))
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [59]:
analise_gasto_parlamentar_19 = tabela_completa.groupby(by=['NomeParlamentar']).sum()[2019].reset_index()
analise_gasto_parlamentar_20 = tabela_completa.groupby(by=['NomeParlamentar']).sum()[2020].reset_index()
analise_gasto_parlamentar_21 = tabela_completa.groupby(by=['NomeParlamentar']).sum()[2021].reset_index()
analise_gasto_parlamentar_22 = tabela_completa.groupby(by=['NomeParlamentar']).sum()[2022].reset_index()

In [60]:
#Analisando os gastos por pardidos nos anos de 2019 até 2022 com a biblioteca plotly
fig = go.Figure()
fig.add_trace(go.Bar(
    x=analise_gasto_parlamentar_19["NomeParlamentar"],
    y=analise_gasto_parlamentar_19[2019],
    name='Gastos 2019',
    marker_color='Orange'
))
fig.add_trace(go.Bar(
    x=analise_gasto_parlamentar_20["NomeParlamentar"],
    y=analise_gasto_parlamentar_20[2020],
    name='Gastos 2020',
    marker_color='Red'
))

fig.add_trace(go.Bar(
    x=analise_gasto_parlamentar_21["NomeParlamentar"],
    y=analise_gasto_parlamentar_21[2021],
    name='Gastos 2021',
    marker_color='Green'
))

fig.add_trace(go.Bar(
    x=analise_gasto_parlamentar_22["NomeParlamentar"],
    y=analise_gasto_parlamentar_22[2022],
    name='Gastos 2022',
    marker_color='Blue'
))
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [61]:
#Analisando os gastos por parlamentar nos ano de 2019 com a biblioteca plotly
grafico = px.bar(analise_gasto_parlamentar_19, 
                 y=2019, 
                 x='NomeParlamentar', 
                 title='Gastos Por Parlamentar em 2019')
grafico.show()

In [62]:
#Analisando os gastos por parlamentar nos ano de 2020 com a biblioteca plotly
grafico = px.bar(analise_gasto_parlamentar_20, 
                 y=2020, 
                 x='NomeParlamentar', 
                 title='Gastos Por Parlamentar em 2020')
grafico.show()

In [63]:
#Analisando os gastos por parlamentar nos ano de 2021 com a biblioteca plotly
grafico = px.bar(analise_gasto_parlamentar_21, 
                 y=2021, 
                 x='NomeParlamentar', 
                 title='Gastos Por Parlamentar em 2021')
grafico.show()

In [65]:
#Analisando os gastos por parlamentar nos ano de 2022 com a biblioteca plotly
grafico = px.bar(analise_gasto_parlamentar_22, 
                 y=2022, 
                 x='NomeParlamentar', 
                 title='Gastos Por Parlamentar em 2022')
grafico.show()

In [68]:
#Agrupar Por ano e por Deputado

analise_ano_deputado_total = filtro.groupby(by=['Ano', 'Matricula']).sum()['Valor'].reset_index()

tabela_completa_1 = pd.merge( base_cadastro[['Matricula','NomeParlamentar','Partido','Email','Telefone']],analise_ano_deputado_total, on='Matricula', how = 'inner')
tabela_completa_1

,Matricula,NomeParlamentar,Partido,Email,Telefone,Ano,Valor
0,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,2019,221749.58
1,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,2020,254126.85
2,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,2021,344173.27
3,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,2022,201652.59
4,300606,Adriana Borgo,AGIR,adrianaborgo@al.sp.gov.br,(11) 3886-6567/8770,2019,215277.49
...,...,...,...,...,...,...,...
355,300430,Vinícius Camarinha,PSDB,depviniciuscamarinha@al.sp.gov.br,3886-6366/6365/6494,2022,190297.15
356,300589,Wellington Moura,REPUBLICANOS,wmoura@al.sp.gov.br,(011)3886-6096 / 6139,2019,287345.95
357,300589,Wellington Moura,REPUBLICANOS,wmoura@al.sp.gov.br,(011)3886-6096 / 6139,2020,270351.36
358,300589,Wellington Moura,REPUBLICANOS,wmoura@al.sp.gov.br,(011)3886-6096 / 6139,2021,360558.44


In [79]:
#Agrupar Por Deputado para analisar qual deputado gastou mais em seu mandato
analise_ano_deputado_total_Nome = tabela_completa_1.groupby(by=['NomeParlamentar']).sum()['Valor'].reset_index()

analise_ano_deputado_total_Nome

,NomeParlamentar,Valor
0,Adalberto Freitas,1021702.29
1,Adriana Borgo,685996.51
2,Agente Federal Danilo Balas,1036993.67
3,Aldo Demarchi,39740.81
4,Alex de Madureira,797725.23
...,...,...
87,Teonilio Barba,1251643.67
88,Thiago Auricchio,883176.70
89,Valeria Bolsonaro,727223.97
90,Vinícius Camarinha,1245854.32


In [80]:
#Cassificando Por parlamentares para analisar qual gastou mais em seu mandato
dados_classificados = analise_ano_deputado_total_Nome.sort_values(["Valor"], ascending = False)
Top_10 = dados_classificados.head(10)

,NomeParlamentar,Valor
36,Enio Tatto,1252262.97
87,Teonilio Barba,1251643.67
90,Vinícius Camarinha,1245854.32
60,Marta Costa,1219451.95
48,Jorge Wilson Xerife do Consumidor,1210910.42
83,Sebastião Santos,1195274.48
66,Márcia Lia,1181812.12
69,Paulo Fiorilo,1178581.78
17,Carlão Pignatari,1155897.94
13,Campos Machado,1153785.94


In [83]:
#Plotando o grafico com os 10 parlamentares que mais gastaram em seu mandato 
grafico = px.bar(top_10, 
                 y='Valor', 
                 x='NomeParlamentar', 
                 title='Gastos Por Parlamentar no Mandato de 2019 até 2022')
grafico.show()

In [84]:
#Fazendo o merge de toda a tabela afim de trazer mais insights
tabela_geral = pd.merge( base_cadastro[['Matricula','NomeParlamentar','Partido','Email','Telefone']],filtro, on='Matricula', how = 'inner')
tabela_geral

,Matricula,NomeParlamentar,Partido,Email,Telefone,Ano,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,2019,3,7200.00,08390859000166,ADALBERTO FREITAS,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",BRITHE GRÁFICA LTDA - EPP
1,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,2019,3,30.00,17781480000178,ADALBERTO FREITAS,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,KREMER & SIMÕES DISTRIBUIDORA DE ÁGUA MINERAL ...
2,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,2019,3,4.00,28720558000160,ADALBERTO FREITAS,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,SHOPPING DO REAL - ANJOS DA ECONOMIA UTILIDADE...
3,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,2019,3,2296.85,43283811003841,ADALBERTO FREITAS,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,KALUNGA COM.E IND. GRÁFICA LTDA.
4,300605,Adalberto Freitas,PSDB,adalbertofreitas@al.sp.gov.br,(11) 3886-6557 / 6559,2019,3,168.75,43283811006867,ADALBERTO FREITAS,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,KALUNGA COM. E IND. GRÁFICA LTDA.
...,...,...,...,...,...,...,...,...,...,...,...,...
56816,300589,Wellington Moura,REPUBLICANOS,wmoura@al.sp.gov.br,(011)3886-6096 / 6139,2022,5,3197.00,09508981000157,WELLINGTON MOURA,N - MORADIA,MF2 EMPREENDIMENTOS IMOBILIARIOS LTDA
56817,300589,Wellington Moura,REPUBLICANOS,wmoura@al.sp.gov.br,(011)3886-6096 / 6139,2022,5,2835.90,02491558000142,WELLINGTON MOURA,O - LOCAÇÃO DE VEÍCULO,UNIDAS VEÍCULOS ESPECIAIS S.A.
56818,300589,Wellington Moura,REPUBLICANOS,wmoura@al.sp.gov.br,(011)3886-6096 / 6139,2022,5,167.32,14394533000100,WELLINGTON MOURA,P - DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR,NUVEM HOSPEDAGEM SOLUÇÕES WEB
56819,300589,Wellington Moura,REPUBLICANOS,wmoura@al.sp.gov.br,(011)3886-6096 / 6139,2022,5,3600.00,33455436000124,WELLINGTON MOURA,P - DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR,"RESOLV SOLUÇÕES EM PRODUÇÃO, COMUNICAÇÃO, EVEN..."


In [89]:
#Cassificando o tipo de gasto do parlamentar que mais gastou em seu mandato para saber onde esse valores foram gastos

analise_ano_deputado_tipo_despesa = tabela_geral.groupby(by=['NomeParlamentar','Tipo']).sum()['Valor'].reset_index()

deputado_com_maior_gasto = analise_ano_deputado_tipo_despesa.loc[(analise_ano_deputado_tipo_despesa['NomeParlamentar']== 'Enio Tatto')]

deputado_com_maior_gasto

,NomeParlamentar,Tipo,Valor
434,Enio Tatto,A - COMBUSTÍVEIS E LUBRIFICANTES,61947.46
435,Enio Tatto,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,86.10
436,Enio Tatto,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",261402.54
437,Enio Tatto,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,47011.81
438,Enio Tatto,F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTOR...,289130.99
439,Enio Tatto,"G - ASSINATURAS DE PERIÓDICOS, PUBLICAÇÕES, IN...",14389.62
440,Enio Tatto,"H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERG...",40045.80
441,Enio Tatto,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",39010.30
442,Enio Tatto,J - SERVIÇOS DE COMUNICAÇÃO,79.00
443,Enio Tatto,K - LOCAÇÃO DE BENS MÓVEIS,95115.81


In [91]:
#Plotando grafico do tipo de gasto do parlamentar que mais gastou em seu mandato para saber onde esse valores foram gastos
grafico = px.bar(deputado_com_maior_gasto, 
                 y='Valor', 
                 x='Tipo', 
                 title='Tipo de gasto do Parlamentar Enio Tatto no Mandato de 2019 até 2022')
grafico.show()